In [1006]:
import urllib 
from bs4 import BeautifulSoup  
import logging  
logging.basicConfig(level=logging.DEBUG)
import string
string.ascii_uppercase
from urllib.request import urlopen
import requests
import re
import pandas as pd

In [945]:
url = 'http://www.boxofficemojo.com/movies/?id=starwars7.htm'

In [946]:
soupURL = url
response = requests.get(soupURL)
page = response.text
soup = BeautifulSoup(page, 'lxml')

In [948]:
import dateutil.parser



def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date

def money_to_int(moneystring):
    moneystring = moneystring.replace('$', '').replace(',', '')
    return int(moneystring)


def runtime_to_minutes(runtimestring):
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None
    
def get_movie_value(soup, field_name):
    '''Grab a value from boxofficemojo HTML
    
    Takes a string attribute of a movie on the page and
    returns the string in the next sibling object
    (the value for that attribute)
    or None if nothing is found.
    '''
    obj = soup.find(text=re.compile(field_name))
    if obj == None: 
        pass
    # this works for most of the values
    next_sibling = obj.findNextSibling()
    if next_sibling:
        return next_sibling.text 

In [1032]:
print(get_movie_value(soup, 'Domestic'))

InvalidSchema: No connection adapters were found for 'html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"'

In [683]:

# Let's get these again and format them all in one swoop
from pprint import pprint

raw_release_date = get_movie_value(soup,'Release Date')
release_date = to_date(raw_release_date)

raw_domestic_total_gross = get_movie_value(soup,'Domestic Total')
domestic_total_gross = money_to_int(raw_domestic_total_gross)

raw_runtime = get_movie_value(soup,'Runtime')
runtime = runtime_to_minutes(raw_runtime)

headers = ['movie title', 'domestic total gross',
           'release date', 'runtime (mins)', 'rating']

movie_data = []
movie_dict = dict(zip(headers, [title,
                                domestic_total_gross,
                                release_date,
                                runtime,
                                rating]))
movie_data.append(movie_dict)

pprint(movie_data)

[{'domestic total gross': 'N/A',
  'movie title': '2015 Yearly Box Office Results - Box Office Mojo',
  'rating': 'R',
  'release date': datetime.datetime(2015, 12, 18, 0, 0),
  'runtime (mins)': 136}]


## Selenium on 2015 table

In [950]:
from selenium import webdriver

In [951]:
driver = webdriver.Firefox()

driver.get('http://www.boxofficemojo.com/yearly/chart/?yr=2015&p=.htm')


In [1043]:
yearly_table = driver.find_element_by_xpath('//*[@id="body"]/table[3]/tbody/tr/td[1]/table/tbody/tr[2]/td/table/tbody/tr[2]/td[4]/font/b')

SyntaxError: invalid syntax (<ipython-input-1043-c1aa02dc9dcc>, line 1)

In [1034]:
yearly_table_html = yearly_table.get_attribute('outerHTML')

In [1035]:
from IPython.core.display import display, HTML

In [1042]:
display(HTML(yearly_table_html))

In [998]:
import lxml.html

root = lxml.html.fromstring(driver.page_source)
for row in root.xpath('.//*[@id="body"]/table[3]/tbody/tr/td[1]/table/tbody/tr[2]/td/table/tbody/tr'):
    cells = row.xpath('.//td')
    dict_value = {'0th': cells[0]}


In [999]:
print(dict_value)

{'0th': <Element td at 0x7fe10638ab88>}


## Get data for the top grossing 100 movies from 2015

In [1014]:
import pandas as pd


In [1015]:
baseURL = ("http://www.boxofficemojo.com/")

In [1030]:
def get_soup(url):
    soupURL = url
    response = requests.get(soupURL)
    page = response.text
    soup = BeautifulSoup(page, 'lxml')
    return soup

In [1017]:
def generate_links(soup):
    my_list = []
    for link in soup.find("body").find_all('a', href=True):
        if link['href'].startswith('/movies/?id='):
            my_list.append(link['href'])
        elif link['href'] == '' or link['href'].startswith('#'):
            continue
    link_list = (my_list[2:])
    return(link_list)


In [1018]:
tables = soup.find('tbody')
tables

In [1024]:
def get_domestic(soup):
    DomesticList = []
    for i in tables:
        DomesticList.append(i)
    print(DomesticList)
    

In [1020]:
def movie_links(links):
    url_set = []
    for link in links:
        url = 'http://www.boxofficemojo.com' + str(link)
        url_set.append(url)
    return url_set

In [1007]:
page2015 = 'http://www.boxofficemojo.com/yearly/chart/?yr=2015&p=.htm'

In [1031]:
links = movie_links(generate_links(get_soup('http://www.boxofficemojo.com/yearly/chart/?page=1&view=releasedate&view2=domestic&yr=2015&p=.htm')))

In [1023]:
get_domestic(get_soup(page2015))

TypeError: 'NoneType' object is not iterable

In [1025]:
def get_movie_value(links, field_name):
    temp = []
    '''Grab a value from boxofficemojo HTML
    
    Takes a string attribute of a movie on the page and returns the string in the next sibling object
    (the value for that attribute)or None if nothing is found. '''
    for url in links:

        soup = get_soup(url)
        obj = soup.find(text=re.compile(field_name))
        if not obj: 
            return None
        # this works for most of the values
        next_sibling = obj.findNextSibling()
        if next_sibling:
            temp.append(next_sibling.text)
    return temp

In [1026]:
def get_title(links):
    titleList = []
    for url in links:
        soup = get_soup(url)
        title_string = soup.find('title').text
        title = title_string.split('(')[0].strip()
        titleList.append(title)
    return titleList

In [1027]:
raw_release_list = get_movie_value(links,'Release Date')
ReleaseDates = []
for i in raw_release_list:
    ReleaseDates.append(to_date(i))
    

TypeError: 'NoneType' object is not iterable

In [1029]:
print(raw_release_list)

None


In [795]:
raw_domestic_total_gross_list = get_movie_value(links,'Domestic Total')
DomesticList = []
for i in raw_domestic_total_gross_list:
    DomesticList.append(money_to_int(i))

In [798]:
raw_runtime_list = get_movie_value(links,'Runtime')
RuntimeList = []
for i in raw_runtime_list:
    RuntimeList.append(runtime_to_minutes(i))

In [800]:
raw_rating_list = get_movie_value(links,'MPAA Rating')
RatingList = []
for i in raw_rating_list:
    RatingList.append(i)


In [ ]:
len(DomesticList)

In [1013]:
d = {'title' : pd.Series(get_title(links), index=range(2,101)),
     #'Total Domestic Growth' : pd.Series(DomesticList, index = range(2,101)),
     'Release Date' : pd.Series(ReleaseList, index = range(2,101)),
     'Runtime' : pd.Series(RuntimeList, index = range(2,101)),
     'Rating' : pd.Series(RatingList, index = range(2,101))}

Exception ignored in: <bound method Service.__del__ of <selenium.webdriver.chrome.service.Service object at 0x7fe106296b70>>
Traceback (most recent call last):
  File "/home/patrick/anaconda3/lib/python3.5/site-packages/selenium/webdriver/common/service.py", line 151, in __del__
    self.stop()
  File "/home/patrick/anaconda3/lib/python3.5/site-packages/selenium/webdriver/common/service.py", line 123, in stop
    if self.process is None:
AttributeError: 'Service' object has no attribute 'process'


NameError: name 'ReleaseList' is not defined

In [1028]:
df = pd.dataframe(d)

AttributeError: module 'pandas' has no attribute 'dataframe'

In [775]:
from pprint import pprint


title = get_title(links)

                 
raw_release_date = get_movie_value(links,'Release Date')
release_date = to_date(raw_release_date)


raw_domestic_total_gross = get_movie_value(links,'Domestic Total')
domestic_total_gross = money_to_int(raw_domestic_total_gross)

raw_runtime = get_movie_value(links,'Runtime')
runtime = runtime_to_minutes(raw_runtime)

raw_rating = get_movie_value(links,'MPAA Rating')
rating = raw_rating


headers = ['movie title', 'domestic total gross',
           'release date', 'runtime (mins)', 'rating']

movie_data = []
movie_dict = dict(zip(headers, [title,
                                domestic_total_gross,
                                release_date,
                                runtime,
                                rating]))
movie_data.append(movie_dict)

pprint(movie_data)


Aloha
2015-06-12 00:00:00
652270625
[{'domestic total gross': 652270625,
  'movie title': 'Aloha',
  'rating': 'PG-13',
  'release date': datetime.datetime(2015, 6, 12, 0, 0),
  'runtime (mins)': 124}]
